In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from pynq import allocate

In [ ]:
from pynq import PL
PL.reset()
import xrfdc

In [ ]:
from pynq import Overlay
ol = Overlay('./argmax.bit')

In [ ]:
# sanity check that we are at 150 MHz
from pynq import Clocks
Clocks.pl_clk0_mhz = 150
print(Clocks.pl_clk0_mhz)

In [ ]:
N = 4096
num_width = 32

In [ ]:
mmio_interface = ol.mmio_interface
dma_out_argmax = ol.axi_dma_0
dma_out_conv = ol.axi_dma_1
dma_prn_config = ol.axi_prn_config
dma_iq_config = ol.axi_iq_config
dma_ifft_config = ol.axi_ifft_config

In [ ]:
# set up ffts
fft_config = allocate(1, dtype=np.uint8)
fft_config[0] = 0
ifft_config = allocate(1, dtype=np.uint8)
ifft_config[0] = 1
# note they are default inverse (1)
dma_prn_config.sendchannel.transfer(fft_config)
dma_prn_config.sendchannel.wait()
dma_iq_config.sendchannel.transfer(fft_config)
dma_iq_config.sendchannel.wait()
dma_ifft_config.sendchannel.transfer(ifft_config)
dma_ifft_config.sendchannel.wait()

In [ ]:
# numerical helpers
def get_twos_from_decimal(decimal, num_bits):
    decimal = int(decimal)
    return decimal % (1<<num_bits)

def get_value_from_complex(real, imag, num_bits):
    return get_twos_from_decimal(real, num_bits)*(1<<num_bits)+get_twos_from_decimal(imag, num_bits)

def get_decimal_from_twos(twos, num_bits):
    twos = int(twos)
    if twos >= (1<<(num_bits-1)):
        return twos - (1<<num_bits)
    else:
        return twos
    
def get_components(complex_array, num_bits):
    return {
        'real': np.array([get_decimal_from_twos(x // (1<<num_bits), num_bits) for x in complex_array]),
        'imag': np.array([get_decimal_from_twos(x % (1<<num_bits), num_bits) for x in complex_array])
    }

In [ ]:
res_data = allocate(2*N, dtype=np.uint32)
def get_conv_data(): #outputs 1 code block
    mmio_interface.write(0x08, 0)
    time.sleep(0.0007)
    mmio_interface.write(0x08, 1)
    dma_out_conv.recvchannel.transfer(res_data)
    dma_out_conv.recvchannel.wait()
    return res_data

In [ ]:
def doppler_sweep(prn_id, start = -8000, end = 8000, step = 100, scaling = 16): #sweeps frequency range
    doppler_range = list(range(start, end, step))
    mmio_interface.write(0x00, scaling)
    mmio_interface.write(0x04, prn_id-1)

    amplitudes = []
    for doppler in doppler_range:
        
#         if doppler % 1000 == 0: print(f'on doppler {doppler}')
            
        # set adc frequency now
        adc_block.MixerSettings['Freq']= center_frequency + doppler / 1e6 # set the frequency of the Numerically controlled oscillator.
        adc_block.UpdateEvent(xrfdc.EVENT_MIXER) #every time setting is changed, must call this.
        
        res_data = get_conv_data()

        # now do calculation
        real = res_data[1::2]
        imag = res_data[0::2]
        real_decimal = [get_decimal_from_twos(x, num_width) for x in real]
        imag_decimal = [get_decimal_from_twos(x, num_width) for x in imag]
        convolve = np.array([x+y*1j for (x,y) in zip(real_decimal, imag_decimal)])
        peak_position = np.argmax(abs(convolve))
        amplitudes.append(abs(convolve)[peak_position]**2)

    plt.plot(doppler_range, amplitudes)
    best_doppler = np.argmax(amplitudes)
    return amplitudes

In [ ]:
def prn_sweep(start = -5000, end = 5000, step = 100, scaling = 16): #sweeps all 37 prns
    full_data = []
    for i in range(1,38):
        amplitudes = doppler_sweep(i,start=start, end=end, step=step, scaling=scaling)
        full_data.append(amplitudes)
        print(f"Frequency sweep of prn ID {i}, max {max(amplitudes)}")        
    return full_data

In [ ]:
# set scaling
mmio_interface.write(0x00, 16)
# set prn = 11
mmio_interface.write(0x04, 10)

In [ ]:
argmax_data = allocate(2*1000, np.int32)

In [ ]:
dma_out_argmax.recvchannel.transfer(argmax_data)
for i in range(1000):
    print(f"on sample {i}")
    get_conv_data()
    dma_out_conv.sendchannel.transfer(res_data)
    dma_out_conv.sendchannel.wait()
#     print(res_data)
# ok, 1000 are done, we expect an output at dma_out_argmax
dma_out_argmax.recvchannel.wait()

In [ ]:
argmax_data

In [ ]:
max(argmax_data)

In [ ]:
indices = argmax_data[0::2]

In [ ]:
plt.hist(indices)